In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pathlib
import os
import glob as gb
import cv2
import PIL
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from keras.models import Model

2024-07-02 15:17:07.491092: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-02 15:17:07.491247: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-02 15:17:07.654743: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
trainpath = '/kaggle/input/riceleafdiseasedataset/dataset/train'
testpath = '/kaggle/input/riceleafdiseasedataset/dataset/test'

# Image Processing

In [3]:
# Image Processing - Training Data
new_size = 224
train_images = []
train_labels = []
class_disease = {'BacterialBlight': 0, 'Blast': 1, 'BrownSpot': 2, 'Tungro': 3}

for i in os.listdir(trainpath):
    if i in class_disease:
        print("Entering the folder:", i)
        files = gb.glob(pathname=str(trainpath + '/' + i + '/*.jpg')) + gb.glob(pathname=str(trainpath + '/' + i + '/*.JPG'))
        print("Number of images in the folder:", len(files))
        for j in files:
            image_raw = cv2.imread(j)
            image = cv2.cvtColor(image_raw, cv2.COLOR_BGR2RGB)
            resize_image = cv2.resize(image, (new_size, new_size))
            train_images.append(list(resize_image))
            train_labels.append(class_disease[i])

# Image Processing - Testing Data
new_size = 224
test_images = []
test_labels = []

for i in os.listdir(testpath):
    if i in class_disease:
        print("Entering to the folder name:", i)
        files = gb.glob(pathname=str(testpath + '/' + i + '/*.jpg')) + gb.glob(pathname=str(testpath + '/' + i + '/*.JPG'))
        print("Number of images in the folder is", len(files))
        for j in files:
            image_raw = cv2.imread(j)
            image = cv2.cvtColor(image_raw, cv2.COLOR_BGR2RGB)
            resize_image = cv2.resize(image, (new_size, new_size))
            test_images.append(list(resize_image))
            test_labels.append(class_disease[i])

def list_to_array_train(train_images, train_labels):
    return np.array(train_images), np.array(train_labels)

X_train, y_train = list_to_array_train(train_images, train_labels)

def list_to_array_test(test_images, test_labels):
    return np.array(test_images), np.array(test_labels)

X_test, y_test = list_to_array_test(test_images, test_labels)

print(X_train.shape)
print("*" * 20)
print(y_train.shape)
print("*" * 20)
print(X_test.shape)
print(y_test.shape)

def keras_to_categorical(y_train, y_test):
    return to_categorical(y_train), to_categorical(y_test)

y_train1 = y_train
y_test1 = y_test
y_train, y_test = keras_to_categorical(y_train, y_test)

y_train1.shape, y_test1.shape

def convert_one_hot_to_categorical(one_hot_labels):
    return np.argmax(one_hot_labels, axis=1)

gpus = tf.config.list_physical_devices('GPU')
print(f"Num GPUs Available: {len(gpus)}")

if len(gpus) < 2:
    print("Not enough GPUs available, ensure your environment is configured correctly")
else:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    strategy = tf.distribute.MirroredStrategy()

Entering the folder: Tungro
Number of images in the folder: 1108
Entering the folder: BacterialBlight
Number of images in the folder: 1384
Entering the folder: Blast
Number of images in the folder: 1240
Entering the folder: BrownSpot
Number of images in the folder: 1400
Entering to the folder name: Tungro
Number of images in the folder is 200
Entering to the folder name: BacterialBlight
Number of images in the folder is 200
Entering to the folder name: Blast
Number of images in the folder is 200
Entering to the folder name: BrownSpot
Number of images in the folder is 200
(5132, 224, 224, 3)
********************
(5132,)
********************
(800, 224, 224, 3)
(800,)
Num GPUs Available: 2


In [4]:
# Function to convert one-hot encoded labels to categorical labels
def convert_one_hot_to_categorical(one_hot_labels):
    return np.argmax(one_hot_labels, axis=1)

In [5]:
# Deep Feature extraction - VGG16
def model_vgg16():
    VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in VGG_model.layers:
        layer.trainable = False
    feature = GlobalAveragePooling2D()(VGG_model.output)
    output = Model(inputs=VGG_model.input, outputs=feature)
    return output

# Deep Feature Extraction - VGG19
def model_vgg19():
    VGG_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in VGG_model.layers:
        layer.trainable = False
    feature = GlobalAveragePooling2D()(VGG_model.output)
    output = Model(inputs=VGG_model.input, outputs=feature)
    return output

# Deep Feature Extraction - ResNet50
def model_resnet50():
    Resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in Resnet_model.layers:
        layer.trainable = False
    feature = GlobalAveragePooling2D()(Resnet_model.output)
    output = Model(inputs=Resnet_model.input, outputs=feature)
    return output

# Deep Feature Extraction - EfficientNet
def model_effnet():
    Effnet_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in Effnet_model.layers:
        layer.trainable = False
    feature = GlobalAveragePooling2D()(Effnet_model.output)
    output = Model(inputs=Effnet_model.input, outputs=feature)
    return output

# Deep Feature Extraction - InceptionV3
def model_inceptionV3():
    Inception_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in Inception_model.layers:
        layer.trainable = False
    feature = GlobalAveragePooling2D()(Inception_model.output)
    output = Model(inputs=Inception_model.input, outputs=feature)
    return output

# VGG16 + VGG19 + Random Forest Classifier

In [6]:
from sklearn.ensemble import RandomForestClassifier

def train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test):
    with strategy.scope():  
        model_FE_16 = model_vgg16()
        model_FE_16.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_16 = model_FE_16.predict(X_train)
        test_feature_16 = model_FE_16.predict(X_test)
        
        model_FE_19 = model_vgg19()
        model_FE_19.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_19 = model_FE_19.predict(X_train)
        test_feature_19 = model_FE_19.predict(X_test)
        
        final_train = np.hstack((train_feature_16, train_feature_19))
        final_test = np.hstack((test_feature_16, test_feature_19))

        rf = RandomForestClassifier()
        rf = rf.fit(final_train, y_train)
        test_pred = rf.predict(final_test)

        accuracy = accuracy_score(y_test, test_pred)
        recall = recall_score(y_test, test_pred, average='weighted')
        precision = precision_score(y_test, test_pred, average='weighted')
        f1 = f1_score(y_test, test_pred, average='weighted')
        auc = roc_auc_score(y_test, test_pred, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Calculate average metrics
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

# Print average metrics
print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Save results to an Excel file
output_file = 'VGG16+VGG19+RF.xlsx'
results_df.to_excel(output_file, index_label='Run')

print(f"\nMetrics results saved to '{output_file}'")


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 22s 92ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step
80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 15s 91ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step
Run 1 - Accuracy: 0.96625000, Recall: 0.96625000, Precision: 0.99873096, F1 Score: 0.98190672, AUC: 0.98291667
161/161 ━━━━━━━━━━━━━━━━━━━━ 13s 78ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 95ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step
Run 2 - Accuracy: 0.96375000, Recall: 0.96375000, Precision: 1.00000000, F1 Score: 0.98128918, AUC: 0.98187500
161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 83ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 81ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 17s 102ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 101ms/step
Run 3 - Accuracy: 0.95625000, Recall: 0.95625000, Precision: 0.99871134, F1 Score: 0.97661869, AUC: 0.97791667
161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 85ms/step
25/25 ━━━━━━━━━━━━━━━━

# VGG16 + VGG19 + Decision Tree Classifier

In [7]:
from sklearn.tree import DecisionTreeClassifier

def train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test):
    with strategy.scope():  
        model_FE_16 = model_vgg16()
        model_FE_16.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_16 = model_FE_16.predict(X_train)
        test_feature_16 = model_FE_16.predict(X_test)
        
        model_FE_19 = model_vgg19()
        model_FE_19.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_19 = model_FE_19.predict(X_train)
        test_feature_19 = model_FE_19.predict(X_test)
        
        final_train = np.hstack((train_feature_16, train_feature_19))
        final_test = np.hstack((test_feature_16, test_feature_19))

        # Define and train Decision Tree Classifier
        dt = DecisionTreeClassifier()
        dt.fit(final_train, y_train)
        test_pred = dt.predict(final_test)

        # Calculate evaluation metrics
        accuracy = accuracy_score(y_test, test_pred)
        recall = recall_score(y_test, test_pred, average='weighted')
        precision = precision_score(y_test, test_pred, average='weighted')
        f1 = f1_score(y_test, test_pred, average='weighted')
        auc = roc_auc_score(y_test, test_pred, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

# Perform multiple runs and store the results
num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Compute average and standard deviation for each metric
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Save results to an Excel file
output_file = 'VGG16+VGG19+DTC.xlsx'
results_df.to_excel(output_file, index_label='Run')

print(f"\nMetrics results saved to '{output_file}'")


161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 83ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 82ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 17s 103ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 102ms/step
Run 1 - Accuracy: 0.92875000, Recall: 0.92875000, Precision: 0.92950096, F1 Score: 0.92878530, AUC: 0.95250000
161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 83ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 17s 102ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 101ms/step
Run 2 - Accuracy: 0.92250000, Recall: 0.92250000, Precision: 0.92355542, F1 Score: 0.92259779, AUC: 0.94833333
161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 83ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 17s 103ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 100ms/step
Run 3 - Accuracy: 0.92125000, Recall: 0.92125000, Precision: 0.92216043, F1 Score: 0.92135746, AUC: 0.94750000
161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 83ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 17s 103ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 101ms/

# VGG16 + VGG19 + KNN Classifier

In [8]:
from sklearn.neighbors import KNeighborsClassifier

def train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test, num):
    with strategy.scope():  
        model_FE_16 = model_vgg16()
        model_FE_16.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_16 = model_FE_16.predict(X_train)
        test_feature_16 = model_FE_16.predict(X_test)
        
        model_FE_19 = model_vgg19()
        model_FE_19.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_19 = model_FE_19.predict(X_train)
        test_feature_19 = model_FE_19.predict(X_test)
        
        final_train = np.hstack((train_feature_16, train_feature_19))
        final_test = np.hstack((test_feature_16, test_feature_19))

        # Define and train KNN Classifier
        knn = KNeighborsClassifier(n_neighbors=num)
        knn.fit(final_train, y_train)
        test_pred = knn.predict(final_test)

        # Calculate evaluation metrics
        accuracy = accuracy_score(y_test, test_pred)
        recall = recall_score(y_test, test_pred, average='weighted')
        precision = precision_score(y_test, test_pred, average='weighted')
        f1 = f1_score(y_test, test_pred, average='weighted')
        auc = roc_auc_score(y_test, test_pred, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

# Perform multiple runs and store the results
num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}
num = 5

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test, num)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")
    num = num+1

# Compute average and standard deviation for each metric
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Save results to an Excel file
output_file = 'VGG16+VGG19+KNN.xlsx'
results_df.to_excel(output_file, index_label='Run')

print(f"\nMetrics results saved to '{output_file}'")


161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 83ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 17s 103ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 101ms/step
Run 1 - Accuracy: 0.98750000, Recall: 0.98750000, Precision: 0.98763571, F1 Score: 0.98746090, AUC: 0.99166667
161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 83ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 82ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 17s 102ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 99ms/step
Run 2 - Accuracy: 0.97875000, Recall: 0.97875000, Precision: 0.98991514, F1 Score: 0.98420822, AUC: 0.98770833
161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 83ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 17s 103ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 102ms/step
Run 3 - Accuracy: 0.98625000, Recall: 0.98625000, Precision: 0.98642416, F1 Score: 0.98623601, AUC: 0.99083333
161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 83ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 17s 102ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 99ms/st

# VGG16 + VGG19 + SVM Classifier

In [9]:
from sklearn.svm import SVC

def train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test):
    with strategy.scope():  
        y_train_cat = convert_one_hot_to_categorical(y_train)
        y_test_cat = convert_one_hot_to_categorical(y_test)
        
        model_FE_16 = model_vgg16()
        model_FE_16.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_16 = model_FE_16.predict(X_train)
        test_feature_16 = model_FE_16.predict(X_test)
        
        model_FE_19 = model_vgg19()
        model_FE_19.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_19 = model_FE_19.predict(X_train)
        test_feature_19 = model_FE_19.predict(X_test)
        
        final_train = np.hstack((train_feature_16, train_feature_19))
        final_test = np.hstack((test_feature_16, test_feature_19))

        # Define and train SVM Classifier
        svm = SVC(probability=True)
        svm.fit(final_train, y_train_cat)
        test_pred = svm.predict(final_test)
        test_pred_proba = svm.predict_proba(final_test)

        # Calculate evaluation metrics
        accuracy = accuracy_score(y_test_cat, test_pred)
        recall = recall_score(y_test_cat, test_pred, average='weighted')
        precision = precision_score(y_test_cat, test_pred, average='weighted')
        f1 = f1_score(y_test_cat, test_pred, average='weighted')
        auc = roc_auc_score(y_test_cat, test_pred_proba, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

# Perform multiple runs and store the results
num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Compute average and standard deviation for each metric
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Save results to an Excel file
output_file = 'VGG16+VGG19+SVM.xlsx'
results_df.to_excel(output_file, index_label='Run')

print(f"\nMetrics results saved to '{output_file}'")


161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 83ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 17s 102ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 98ms/step
Run 1 - Accuracy: 1.00000000, Recall: 1.00000000, Precision: 1.00000000, F1 Score: 1.00000000, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 83ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 18s 106ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 105ms/step
Run 2 - Accuracy: 1.00000000, Recall: 1.00000000, Precision: 1.00000000, F1 Score: 1.00000000, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 83ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 17s 104ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 105ms/step
Run 3 - Accuracy: 1.00000000, Recall: 1.00000000, Precision: 1.00000000, F1 Score: 1.00000000, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 83ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 17s 105ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 103ms/s

# VGG16 + VGG19 + XGBoost Classifier

In [12]:
from xgboost import XGBClassifier

def train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test):
    with strategy.scope():  
        y_train_cat = convert_one_hot_to_categorical(y_train)
        y_test_cat = convert_one_hot_to_categorical(y_test)
        
        model_FE_16 = model_vgg16()
        model_FE_16.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_16 = model_FE_16.predict(X_train)
        test_feature_16 = model_FE_16.predict(X_test)
        
        model_FE_19 = model_vgg19()
        model_FE_19.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_19 = model_FE_19.predict(X_train)
        test_feature_19 = model_FE_19.predict(X_test)
        
        final_train = np.hstack((train_feature_16, train_feature_19))
        final_test = np.hstack((test_feature_16, test_feature_19))

        # Define and train XGB Classifier
        xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
        xgb.fit(final_train, y_train_cat)
        test_pred = xgb.predict(final_test)
        test_pred_proba = xgb.predict_proba(final_test)

        # Calculate evaluation metrics
        accuracy = accuracy_score(y_test_cat, test_pred)
        recall = recall_score(y_test_cat, test_pred, average='weighted')
        precision = precision_score(y_test_cat, test_pred, average='weighted')
        f1 = f1_score(y_test_cat, test_pred, average='weighted')
        auc = roc_auc_score(y_test_cat, test_pred_proba, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

# Perform multiple runs and store the results
num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Compute average and standard deviation for each metric
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Save results to an Excel file
output_file = 'VGG16+VGG19+XGB.xlsx'
results_df.to_excel(output_file, index_label='Run')

print(f"\nMetrics results saved to '{output_file}'")


161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 83ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 18s 105ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 104ms/step
Run 1 - Accuracy: 0.99375000, Recall: 0.99375000, Precision: 0.99379341, F1 Score: 0.99374984, AUC: 0.99998750
161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 83ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 17s 104ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 106ms/step
Run 2 - Accuracy: 0.99375000, Recall: 0.99375000, Precision: 0.99379341, F1 Score: 0.99374984, AUC: 0.99998750
161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 83ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 17s 105ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 105ms/step
Run 3 - Accuracy: 0.99375000, Recall: 0.99375000, Precision: 0.99379341, F1 Score: 0.99374984, AUC: 0.99998750
161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 83ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 17s 105ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 106ms/